In [1]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
name="AthMethPre_crossvalidation"

In [2]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [3]:
"""
    cross validation
"""

datapath ="/home02/chenhuangrong/AthMethPre.csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
print clf.best_score_
gamma=clf.best_params_['gamma']
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1)
ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/'+name+'_predict.csv',header=None,index=False)
ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
# easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home02/chenhuangrong/cross_validation_'+name+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

0.736036725325
[[['svmC:0.561231024155gamma:0.03125', 0.73603672532517217, 0.7382239382239382, 0.7314460596786534, 0.7314460596786534, 0.740627390971691, 0.7360224091808107, 0.73481936971560324, 0.73481936971560324, 0.47209334905946976, '0.7360', 956.0, 351.0, 339.0, 968.0, 1307.0, 1307.0]]]
540


In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV

y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
seq=[]
m6a_2614_sequence="/home02/chenhuangrong/AthMethPre.csv"
RNA_code='ACGU'
k=1
interval=1
divided_num=10.0
division_num=10
seq=pd.read_csv(m6a_2614_sequence,header=None,index_col=None)
seq=seq.values
positive_seq=seq[0:1307,:]
# X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_data, train_target, test_size=0.1, random_state=0)
negative_seq=seq[1307:2614,:]
kf = KFold(n_splits=division_num,shuffle=False)  
for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
    
    X_train = np.array(positive_negative_x_train)
    Y_train = list(map(lambda x: 1, xrange(len(positive_negative_x_train) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(positive_negative_x_train) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(positive_negative_y_train)
    Y_test  = list(map(lambda x: 1, xrange(len(positive_negative_y_train) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(positive_negative_y_train) / 2)))
    Y_test.extend(Y2_test )
    Y_test  = np.array(Y_test)
    print "train and test samples finished"
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,14)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,14))}
    print "begin grid search"
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    C=clf.best_params_['C']
    y_pred_prob=clf.predict_proba(X_test)
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    y_pred=clf.predict(X_test)
    
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
#     ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)
    
    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/AthMethPre_independent_test__.csv',header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all,pos_all,neg_all]]]
print savedata
easy_excel.save("tenfold_crossvalidation",[str(X_train.shape[1])],savedata,'/home02/chenhuangrong/AthMethPre_independent_test__.xls')

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


train and test samples finished
begin grid search
c: 1.1125314761 gamma: 0.0659228797801
0.767175572519
ACC_all: 0.767175572519
train and test samples finished
begin grid search
c: 1.1125314761 gamma: 0.0659228797801
0.75572519084
ACC_all: 1.52290076336
train and test samples finished
begin grid search
c: 0.765983178668 gamma: 0.0453882142536
0.80534351145
ACC_all: 2.32824427481
train and test samples finished
begin grid search
c: 0.363105714029 gamma: 0.0453882142536
0.706106870229
ACC_all: 3.03435114504
train and test samples finished
begin grid search
c: 0.363105714029 gamma: 0.03125
0.702290076336
ACC_all: 3.73664122137
train and test samples finished
begin grid search
c: 0.527383038241 gamma: 0.03125
0.74427480916
ACC_all: 4.48091603053
train and test samples finished
begin grid search
c: 1.1125314761 gamma: 0.03125
0.74427480916
ACC_all: 5.22519083969
train and test samples finished
begin grid search
c: 0.363105714029 gamma: 0.03125
0.738461538462
ACC_all: 5.96365237816
train and

True